In [1]:
import os
os.environ["XDG_CACHE_HOME"] = "/home/olab/tomerronen1/xdg_cache"
os.environ["AUTH_TOKEN"] = "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"
import datasets
dataset_dict = datasets.load_dataset("hotpot_qa", "distractor")


Reusing dataset hotpot_qa (/home/olab/tomerronen1/xdg_cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/133b9501f892e5193babbad937bee3b4899deb4691ef4d791e6ac0111c875bb5)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
example = dataset_dict["train"][6]
print(example["question"])
print(example["answer"])
print(example["supporting_facts"])
for title, sentence_id in zip(example["supporting_facts"]["title"], example["supporting_facts"]["sent_id"]):
    paragraph_id = example["context"]["title"].index(title)
    print(example["context"]["sentences"][paragraph_id][sentence_id])

Which genus of moth in the world's seventh-largest country contains only one species?
Crambidae
{'title': ['Indogrammodes', 'Indogrammodes', 'India', 'India'], 'sent_id': [0, 1, 0, 1]}
Indogrammodes is a genus of moths of the Crambidae family.
 It contains only one species, Indogrammodes pectinicornalis, which is found in India.
India, officially the Republic of India ("Bhārat Gaṇarājya"), is a country in South Asia.
 It is the seventh-largest country by area, the second-most populous country (with over 1.2 billion people), and the most populous democracy in the world.


In [20]:
from transformers.file_utils import cached_path

ImportError: cannot import name 'cached_path' from 'transformers.file_utils' (/home/olab/tomerronen1/miniconda3/envs/mlskel/lib/python3.9/site-packages/transformers/file_utils.py)

In [17]:
S3_BUCKET_PREFIX = "https://s3.amazonaws.com/models.huggingface.co/bert"
CLOUDFRONT_DISTRIB_PREFIX = "https://cdn.huggingface.co"


def hf_bucket_url(model_id: str, filename: str, use_cdn=True) -> str:
    """
    Taken from https://github.com/huggingface/transformers/blob/1bc13697b1d2197a18f4ed6009e19aed315ab0f0/src/transformers/file_utils.py#L451
    """
    endpoint = CLOUDFRONT_DISTRIB_PREFIX if use_cdn else S3_BUCKET_PREFIX
    legacy_format = "/" not in model_id
    if legacy_format:
        return f"{endpoint}/{model_id}-{filename}"
    else:
        return f"{endpoint}/{model_id}/{filename}"


from typing import Union, Dict, Optional
from pathlib import Path
from urllib.parse import urlparse
from transformers.file_utils import TRANSFORMERS_CACHE




def is_remote_url(url_or_filename):
    parsed = urlparse(url_or_filename)
    return parsed.scheme in ("http", "https")

def cached_path(
    url_or_filename,
    cache_dir=None,
    force_download=False,
    proxies=None,
    resume_download=False,
    user_agent: Union[Dict, str, None] = None,
    extract_compressed_file=False,
    force_extract=False,
    local_files_only=False,
) -> Optional[str]:
    """
    Taken from https://github.com/huggingface/transformers/blob/1bc13697b1d2197a18f4ed6009e19aed315ab0f0/src/transformers/file_utils.py#L525
    """
    if cache_dir is None:
        cache_dir = TRANSFORMERS_CACHE
    if isinstance(url_or_filename, Path):
        url_or_filename = str(url_or_filename)
    if isinstance(cache_dir, Path):
        cache_dir = str(cache_dir)

    if is_remote_url(url_or_filename):
        # URL, so get it from the cache (downloading if necessary)
        output_path = get_from_cache(
            url_or_filename,
            cache_dir=cache_dir,
            force_download=force_download,
            proxies=proxies,
            resume_download=resume_download,
            user_agent=user_agent,
            local_files_only=local_files_only,
        )
    elif os.path.exists(url_or_filename):
        # File, and it exists.
        output_path = url_or_filename
    elif urlparse(url_or_filename).scheme == "":
        # File, but it doesn't exist.
        raise EnvironmentError("file {} not found".format(url_or_filename))
    else:
        # Something unknown
        raise ValueError("unable to parse {} as a URL or as a local path".format(url_or_filename))

    if extract_compressed_file:
        if not is_zipfile(output_path) and not tarfile.is_tarfile(output_path):
            return output_path

        # Path where we extract compressed archives
        # We avoid '.' in dir name and add "-extracted" at the end: "./model.zip" => "./model-zip-extracted/"
        output_dir, output_file = os.path.split(output_path)
        output_extract_dir_name = output_file.replace(".", "-") + "-extracted"
        output_path_extracted = os.path.join(output_dir, output_extract_dir_name)

        if os.path.isdir(output_path_extracted) and os.listdir(output_path_extracted) and not force_extract:
            return output_path_extracted

        # Prevent parallel extractions
        lock_path = output_path + ".lock"
        with FileLock(lock_path):
            shutil.rmtree(output_path_extracted, ignore_errors=True)
            os.makedirs(output_path_extracted)
            if is_zipfile(output_path):
                with ZipFile(output_path, "r") as zip_file:
                    zip_file.extractall(output_path_extracted)
                    zip_file.close()
            elif tarfile.is_tarfile(output_path):
                tar_file = tarfile.open(output_path)
                tar_file.extractall(output_path_extracted)
                tar_file.close()
            else:
                raise EnvironmentError("Archive format of {} could not be identified".format(output_path))

        return output_path_extracted

    return output_path



import transformers
transformers.utils.hf_bucket_url = hf_bucket_url
import sled

ImportError: cannot import name 'cached_path' from 'transformers.utils' (/home/olab/tomerronen1/miniconda3/envs/mlskel/lib/python3.9/site-packages/transformers/utils/__init__.py)

In [13]:
import sled
from transformers import AutoModelForSeq2SeqLM, AutoConfig
model = AutoModelForSeq2SeqLM.from_pretrained("tau/bart-base-sled",
                                              use_auth_token="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")


ImportError: cannot import name 'hf_bucket_url' from 'transformers.utils' (/home/olab/tomerronen1/miniconda3/envs/mlskel/lib/python3.9/site-packages/transformers/utils/__init__.py)